In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

random_state = 42
load_path = 'KaggleData/'
save_path = 'PreprocessedData/'

### Load Data

In [42]:
application_df = pd.read_csv(load_path + 'application_record.csv')
print(application_df.columns)
application_df

Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS'],
      dtype='object')


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [43]:
for column in application_df.columns:
    print(f"Value counts for column {column}:")
    print(application_df[column].value_counts(dropna=False))
    print("\n")

Value counts for column ID:
ID
7702516    2
7602432    2
7836971    2
7213374    2
7052783    2
          ..
6840104    1
6840222    1
6841878    1
6842765    1
5008808    1
Name: count, Length: 438510, dtype: int64


Value counts for column CODE_GENDER:
CODE_GENDER
F    294440
M    144117
Name: count, dtype: int64


Value counts for column FLAG_OWN_CAR:
FLAG_OWN_CAR
N    275459
Y    163098
Name: count, dtype: int64


Value counts for column FLAG_OWN_REALTY:
FLAG_OWN_REALTY
Y    304074
N    134483
Name: count, dtype: int64


Value counts for column CNT_CHILDREN:
CNT_CHILDREN
0     304071
1      88527
2      39884
3       5430
4        486
5        133
7          9
9          5
12         4
6          4
14         3
19         1
Name: count, dtype: int64


Value counts for column AMT_INCOME_TOTAL:
AMT_INCOME_TOTAL
135000.0    46879
157500.0    39768
180000.0    37912
112500.0    36979
225000.0    34130
            ...  
45450.0         1
403200.0        1
93991.5         1
36828.0      

In [44]:
# Convert binary categorical columns to numerical values
binary_columns = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
binary_mappings = {
    'CODE_GENDER': {'M': 1, 'F': 0},
    'FLAG_OWN_CAR': {'Y': 1, 'N': 0},
    'FLAG_OWN_REALTY': {'Y': 1, 'N': 0}
}

for col in binary_columns:
    application_df[col] = application_df[col].map(binary_mappings[col])

# Rename CODE_GENDER to FLAG_MALE for consistency
application_df.rename(columns={'CODE_GENDER': 'FLAG_MALE'}, inplace=True)

In [45]:
# Create FLAG_EMPLOYED column
application_df['FLAG_EMPLOYED'] = np.where(application_df['OCCUPATION_TYPE'].isnull(), 0, 1)
print(application_df['FLAG_EMPLOYED'].value_counts())

FLAG_EMPLOYED
1    304354
0    134203
Name: count, dtype: int64


In [46]:
# Remove FLAG_MOBIL column since it is always True (1)
application_df.drop(columns=['FLAG_MOBIL'], inplace=True)

In [47]:
# Reserse the sign of DAYS_BIRTH to make it positive
application_df['DAYS_AGE'] = application_df['DAYS_BIRTH'] * -1
application_df.drop(columns=['DAYS_BIRTH'], inplace=True)

In [48]:
# Create FLAG_CHILDREN column
application_df['FLAG_CHILDREN'] = np.where(application_df['CNT_CHILDREN'] > 0, 1, 0)
print(application_df['FLAG_CHILDREN'].value_counts())

FLAG_CHILDREN
0    304071
1    134486
Name: count, dtype: int64


In [49]:
application_df.info()
application_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   FLAG_MALE            438557 non-null  int64  
 2   FLAG_OWN_CAR         438557 non-null  int64  
 3   FLAG_OWN_REALTY      438557 non-null  int64  
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_EMPLOYED        438557 non-null  int64  
 11  FLAG_WORK_PHONE      438557 non-null  int64  
 12  FLAG_PHONE           438557 non-null  int64  
 13  FLAG_EMAIL           438557 non-null  int64  
 14  OCCUPATION_TYPE      304354 non-null  object 
 15  CNT_FAM_MEMBERS  

,ID,FLAG_MALE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,FLAG_EMPLOYED,DAYS_AGE,FLAG_CHILDREN
count,4.385570e+05,438557.000000,438557.000000,438557.000000,438557.000000,4.385570e+05,438557.000000,438557.000000,438557.000000,438557.000000,438557.000000,438557.000000,438557.000000,438557.000000
mean,6.022176e+06,0.328616,0.371897,0.693351,0.427390,1.875243e+05,60563.675328,0.206133,0.287771,0.108207,2.194465,0.693990,15997.904649,0.306656
std,5.716370e+05,0.469711,0.483312,0.461103,0.724882,1.100869e+05,138767.799647,0.404527,0.452724,0.310642,0.897207,0.460835,4185.030007,0.461106
min,5.008804e+06,0.000000,0.000000,0.000000,0.000000,2.610000e+04,-17531.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7489.000000,0.000000
25%,5.609375e+06,0.000000,0.000000,0.000000,0.000000,1.215000e+05,-3103.000000,0.000000,0.000000,0.000000,2.000000,0.000000,12514.000000,0.000000
50%,6.047745e+06,0.000000,0.000000,1.000000,0.000000,1.607805e+05,-1467.000000,0.000000,0.000000,0.000000,2.000000,1.000000,15630.000000,0.000000
75%,6.456971e+06,1.000000,1.000000,1.000000,1.000000,2.250000e+05,-371.000000,0.000000,1.000000,0.000000,3.000000,1.000000,19483.000000,1.000000
max,7.999952e+06,1.000000,1.000000,1.000000,19.000000,6.750000e+06,365243.000000,1.000000,1.000000,1.000000,20.000000,1.000000,25201.000000,1.000000


In [50]:
credit_df = pd.read_csv(load_path + 'credit_record.csv')
credit_df

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [51]:
credit_df = credit_df.drop(columns=['MONTHS_BALANCE'])

In [52]:
for column in credit_df.columns:
    print(f"Value counts for column {column}:")
    print(credit_df[column].value_counts(dropna=False))
    print("\n")

Value counts for column ID:
ID
5001730    61
5023644    61
5056075    61
5058369    61
5011920    61
           ..
5026238     1
5092123     1
5096818     1
5092196     1
5092139     1
Name: count, Length: 45985, dtype: int64


Value counts for column STATUS:
STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64




In [53]:
credit_df['FLAG_PASTDUE'] = credit_df['STATUS'].apply(lambda x: 1 if x in {'0', '1', '2', '3', '4', '5'} else 0)

print(credit_df['FLAG_PASTDUE'].value_counts())

FLAG_PASTDUE
0    651261
1    397314
Name: count, dtype: int64


In [54]:
# Group by ID and calculate the average FLAG_PASTDUE
flag_pastdue_avg = credit_df.groupby('ID')['FLAG_PASTDUE'].mean().reset_index()

# Create FLAG_APPROVED column based on the condition
flag_pastdue_avg['FLAG_APPROVED'] = flag_pastdue_avg['FLAG_PASTDUE'].apply(lambda x: 0 if x >= 0.5 else 1)

flag_pastdue_avg['FLAG_APPROVED'].value_counts()

FLAG_APPROVED
1    24656
0    21329
Name: count, dtype: int64

In [55]:
flag_pastdue_avg.info()
flag_pastdue_avg.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45985 entries, 0 to 45984
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             45985 non-null  int64  
 1   FLAG_PASTDUE   45985 non-null  float64
 2   FLAG_APPROVED  45985 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 1.1 MB


,ID,FLAG_PASTDUE,FLAG_APPROVED
count,4.598500e+04,45985.000000,45985.000000
mean,5.070164e+06,0.475842,0.536175
std,4.543364e+04,0.354110,0.498695
min,5.001711e+06,0.000000,0.000000
25%,5.026147e+06,0.155172,0.000000
50%,5.065737e+06,0.428571,1.000000
75%,5.114024e+06,0.833333,1.000000
max,5.150487e+06,1.000000,1.000000


In [56]:
dataset = pd.merge(application_df, flag_pastdue_avg[['ID', 'FLAG_APPROVED']], on='ID', how='inner')
dataset


,ID,FLAG_MALE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,FLAG_EMPLOYED,DAYS_AGE,FLAG_CHILDREN,FLAG_APPROVED
0,5008804,1,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542,1,0,0,NaN,2.0,0,12005,0,1
1,5008805,1,1,1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542,1,0,0,NaN,2.0,0,12005,0,1
2,5008806,1,1,1,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-1134,0,0,0,Security staff,2.0,1,21474,0,1
3,5008808,0,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051,0,1,1,Sales staff,1.0,1,19110,0,1
4,5008809,0,0,1,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051,0,1,1,Sales staff,1.0,1,19110,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,1,1,1,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-2420,0,0,0,Managers,2.0,1,17348,0,1
36453,5149834,0,0,1,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-1325,0,1,1,Medicine staff,2.0,1,12387,0,0
36454,5149838,0,0,1,0,157500.0,Pensioner,Higher education,Married,House / apartment,-1325,0,1,1,Medicine staff,2.0,1,12387,0,0
36455,5150049,0,0,1,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-655,0,0,0,Sales staff,2.0,1,17958,0,0


In [57]:
dataset.info()
dataset.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   36457 non-null  int64  
 1   FLAG_MALE            36457 non-null  int64  
 2   FLAG_OWN_CAR         36457 non-null  int64  
 3   FLAG_OWN_REALTY      36457 non-null  int64  
 4   CNT_CHILDREN         36457 non-null  int64  
 5   AMT_INCOME_TOTAL     36457 non-null  float64
 6   NAME_INCOME_TYPE     36457 non-null  object 
 7   NAME_EDUCATION_TYPE  36457 non-null  object 
 8   NAME_FAMILY_STATUS   36457 non-null  object 
 9   NAME_HOUSING_TYPE    36457 non-null  object 
 10  DAYS_EMPLOYED        36457 non-null  int64  
 11  FLAG_WORK_PHONE      36457 non-null  int64  
 12  FLAG_PHONE           36457 non-null  int64  
 13  FLAG_EMAIL           36457 non-null  int64  
 14  OCCUPATION_TYPE      25134 non-null  object 
 15  CNT_FAM_MEMBERS      36457 non-null 

,ID,FLAG_MALE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,FLAG_EMPLOYED,DAYS_AGE,FLAG_CHILDREN,FLAG_APPROVED
count,3.645700e+04,36457.000000,36457.000000,36457.000000,36457.000000,3.645700e+04,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000
mean,5.078227e+06,0.329895,0.379708,0.672189,0.430315,1.866857e+05,59262.935568,0.225526,0.294813,0.089722,2.198453,0.689415,15975.173382,0.308747,0.518940
std,4.187524e+04,0.470181,0.485321,0.469422,0.742367,1.017892e+05,137651.334859,0.417934,0.455965,0.285787,0.911686,0.462740,4200.549944,0.461983,0.499648
min,5.008804e+06,0.000000,0.000000,0.000000,0.000000,2.700000e+04,-15713.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7489.000000,0.000000,0.000000
25%,5.042028e+06,0.000000,0.000000,0.000000,0.000000,1.215000e+05,-3153.000000,0.000000,0.000000,0.000000,2.000000,0.000000,12462.000000,0.000000,0.000000
50%,5.074614e+06,0.000000,0.000000,1.000000,0.000000,1.575000e+05,-1552.000000,0.000000,0.000000,0.000000,2.000000,1.000000,15563.000000,0.000000,1.000000
75%,5.115396e+06,1.000000,1.000000,1.000000,1.000000,2.250000e+05,-408.000000,0.000000,1.000000,0.000000,3.000000,1.000000,19438.000000,1.000000,1.000000
max,5.150487e+06,1.000000,1.000000,1.000000,19.000000,1.575000e+06,365243.000000,1.000000,1.000000,1.000000,20.000000,1.000000,25152.000000,1.000000,1.000000


In [58]:
dataset.drop(columns=['ID'], inplace=True)

In [59]:
# Replace 0/1 with False/True for columns starting with 'FLAG_'
flag_columns = [col for col in dataset.columns if col.startswith('FLAG_')]
dataset[flag_columns] = dataset[flag_columns].replace({0: False, 1: True})
dataset

C:\Users\sheam\AppData\Local\Temp\ipykernel_8760\3567563860.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset[flag_columns] = dataset[flag_columns].replace({0: False, 1: True})


,FLAG_MALE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,FLAG_EMPLOYED,DAYS_AGE,FLAG_CHILDREN,FLAG_APPROVED
0,True,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542,True,False,False,NaN,2.0,False,12005,False,True
1,True,True,True,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542,True,False,False,NaN,2.0,False,12005,False,True
2,True,True,True,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-1134,False,False,False,Security staff,2.0,True,21474,False,True
3,False,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051,False,True,True,Sales staff,1.0,True,19110,False,True
4,False,False,True,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051,False,True,True,Sales staff,1.0,True,19110,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,True,True,True,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-2420,False,False,False,Managers,2.0,True,17348,False,True
36453,False,False,True,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-1325,False,True,True,Medicine staff,2.0,True,12387,False,False
36454,False,False,True,0,157500.0,Pensioner,Higher education,Married,House / apartment,-1325,False,True,True,Medicine staff,2.0,True,12387,False,False
36455,False,False,True,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-655,False,False,False,Sales staff,2.0,True,17958,False,False


In [60]:
# Split the dataset into train and test sets
train_data, test_data = train_test_split(dataset, test_size=0.3, random_state=random_state)

# Save the train and test sets to CSV files
train_data.to_csv(save_path + 'Train_data.csv', index=False)
test_data.to_csv(save_path + 'Test_data.csv', index=False)

In [61]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25519 entries, 30988 to 15795
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FLAG_MALE            25519 non-null  bool   
 1   FLAG_OWN_CAR         25519 non-null  bool   
 2   FLAG_OWN_REALTY      25519 non-null  bool   
 3   CNT_CHILDREN         25519 non-null  int64  
 4   AMT_INCOME_TOTAL     25519 non-null  float64
 5   NAME_INCOME_TYPE     25519 non-null  object 
 6   NAME_EDUCATION_TYPE  25519 non-null  object 
 7   NAME_FAMILY_STATUS   25519 non-null  object 
 8   NAME_HOUSING_TYPE    25519 non-null  object 
 9   DAYS_EMPLOYED        25519 non-null  int64  
 10  FLAG_WORK_PHONE      25519 non-null  bool   
 11  FLAG_PHONE           25519 non-null  bool   
 12  FLAG_EMAIL           25519 non-null  bool   
 13  OCCUPATION_TYPE      17589 non-null  object 
 14  CNT_FAM_MEMBERS      25519 non-null  float64
 15  FLAG_EMPLOYED        25519 non-null  